# Modified 3D PMRNN

## Module Imports

In [1]:
from Reconstruction import Reconstruction
from CONFIGURATION import CONFIGURATION
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [2]:
# ignorable
print("TensorFlow version:", tf.__version__)
print("cv2 version:", cv2.__version__)
print("np version:", np.__version__)

TensorFlow version: 2.7.0
cv2 version: 4.5.5
np version: 1.19.5


In [3]:
# ignorable
import sklearn
import matplotlib
print("matplotlib version:", matplotlib.__version__)
print("sklearn version:", sklearn.__version__)

matplotlib version: 3.6.3
sklearn version: 1.2.2


In [ ]:
from tensorflow.keras import layers, losses  # ,Sequential,metrics
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
# from tensorflow.keras.layers import Layer
# from tensorflow.keras import optimizers
# from tensorflow.keras.optimizers.experimental import SGD
# from tensorflow.image import ssim

## Load new Data

### reading data

In [ ]:
import Utils
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape

### Voxel extraction 

In [ ]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

In [ ]:
del image_data

### Train and test data extraction

In [ ]:
X_train = voxels[:-6]
X_test = voxels[-6:]

In [ ]:
del voxels

In [ ]:
len(X_train)

### Save Traited data for uses

In [ ]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [ ]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

## TensorFlow GPU setup

In [4]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
device = tf.config.list_physical_devices("GPU")
print(device)  # Check if GPU devices are visible
# Check if TensorFlow is built with CUDA support
print(tf.test.is_built_with_cuda())
print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True
(7, 2, 2)


In [5]:
# only on weak GPU
tf.config.experimental.set_memory_growth(device[0], True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [6]:
tf.keras.backend.clear_session()

## Load Traited data for training (voxels)

In [82]:
X_train = np.load('X_train.npy')
X_train.shape

(58, 250, 256, 256, 1)

In [ ]:
# for test only
X_train = X_train[:4]
with open('X_train_test.npy', 'wb') as f:
    np.save(f, X_train)
X_train.shape

In [7]:
X_train = np.load('X_train_test.npy')
X_train.shape

(4, 250, 256, 256, 1)

### idk

In [83]:
learnedVoxels = []
inferenceVoxels = []
for voxel in X_train:
    learnedVoxels.append(voxel[:-1])
    inferenceVoxels.append(voxel[1:])

# learnedVoxels = np.array(learnedVoxels)
# inferenceVoxels = np.array(inferenceVoxels)

## Training 

### Imports

In [9]:
from mealpy.swarm_based import GWO
from sklearn.preprocessing import LabelEncoder
import time
import pickle
from datetime import datetime

In [10]:
import mealpy
mealpy.__version__

'2.4.2'

### Lables

In [61]:
class Timer():
    def __init__(self, title=" ", p={}) -> None:
        self.title = title
        self.started = time.time()
        self.params = p
        print(f"{self.title} Started at : {datetime.fromtimestamp(self.started)}")
        self.ended = None

    def End(self):
        self.ended = time.time()
        print(f"{self.title} Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s")

    def setParams(self, name, data):
        self.params[name] = data

    def toDuration(self):
        if self.ended == None:
            return -1
        return self.ended-self.started

    def __str__(self) -> str:
        return f"{self.title} Started at : {datetime.fromtimestamp(self.started)} | Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s"


class TrainingData(Timer):
    def __init__(self, title=" ", p={}) -> None:
        super().__init__(title=title, p=p)
        self.samples = []

    def SampleStarted(self, title=" ") -> Timer:
        timer = Timer(title)
        self.samples.append(timer)
        return timer

    def __str__(self) -> str:
        return super().__str__()+" "+str(self.params)

In [62]:
OPT_ENCODER = LabelEncoder()
# OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'])
OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adam'])
trainings_duration = []
trainings_epochs = []
current_iteration = 0

### Solution Decodation function

In [63]:
def decode_solution(solution):
    batch_size = 2**int(solution[0])

    learning_rate = solution[1]

    opt_int = int(solution[2])
    opt = OPT_ENCODER.inverse_transform([opt_int])[0]

    epoch = 10 * int(solution[3])

    num_filters = int(solution[4])

    latent_space_dim = 2**int(solution[5])

    reduced_dim = 2**int(solution[6])

    return [batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim]

### Objective Function

In [64]:
""" import random
# the range of float values
start = 0.001
end = 0.8 """
current_training=0

def objective_function(solution):
    tf.keras.backend.clear_session()
    global current_training
    print("\n###########################################################\n")
    print(f"\n--------------------------{current_iteration}/{current_training}--------------------------------\n")
    print("\n###########################################################\n")
    batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = decode_solution(solution)
    reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                    latent_space_dim=latent_space_dim,
                                    reducedDimension=reduced_dim,
                                    num_conv_layers=num_filters,
                                    learning_rate=learning_rate,
                                    batch_size=batch_size,
                                    epochs=epoch,
                                    opt=opt,
                                    )
    # reconstruction.summary()
    reconstruction.compile()
    
    params = {
        "inputShape": CONFIGURATION["INPUT_SHAPE"],
        "latent_space_dim": latent_space_dim,
        "reducedDimension": reduced_dim,
        "num_conv_layers": num_filters,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epoch,
        "opt": opt,
        "finished":False
    }
    trainingTime = TrainingData(f"Epoch Training {current_training}", params)
    trainings_duration.append(trainingTime)
    training_loss=1
    for i in range(len(learnedVoxels)):
        print(f"---Currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        sampleTime = trainingTime.SampleStarted(f"Voxel Training {i+1}")
        try:
            # Code that may raise an OOM error
            training_result=reconstruction.train(learnedVoxels[i], inferenceVoxels[i])
            sampleTime.End()
            training_loss=training_result.history['loss'][-1]
            print(training_loss)
            if (training_loss > 0.01) or (i==1 and training_loss > 0.009)or (i>1 and training_loss > 0.008):
                trainingTime.setParams("Bad_params",True)
                break
        except MemoryError as e:
            # Code to handle the OOM error
            trainingTime.setParams("OMM_Errored",str(e))
            break
        except Exception as e:
            trainingTime.setParams("Error",str(e))
            break
        

    trainingTime.setParams("loss",training_loss)
    trainingTime.setParams("finished",True)
    trainingTime.End()

    current_training+=1
    
    """ reconstruction.save( save_folder=f"results/model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}") """
    #return min(histories, key=lambda x: float('inf') if x is None else x)
    return training_loss

### Configurations

In [65]:
LB = [3, 0.001, 0, 2,  4,  6, 7]
UB = [5.99, 0.01, 3.99, 4.99,  8.99,  8.99, 9.99]

problem_dict = {
    "fit_func": objective_function,
    "lb": LB,
    "ub": UB,
    "minmax": "min",
    "verbose": True,
}
metaheuristic_configuration = {
    "epoch": 5,
    "pop_size": 10
}
# save session for x epoch of the training
save_interval = 1
session_file = 'gwo_session.pkl'

### Lunching the metaheuristic

In [45]:
# Check if a session file exists
if os.path.exists(session_file):
    # Load the session and resume from the saved state
    with open(session_file, 'rb') as f:
        session_data = pickle.load(f)
    model = session_data['model']
    best_solution = session_data['best_solution']
    current_iteration = session_data['current_iteration']
    print(best_solution)
    print("Resuming from iteration", current_iteration)
    # raise ValueError("i just wanted to stop")
else:
    # Create a new GWO optimization session
    model = GWO.BaseGWO(
        problem_dict, epoch=metaheuristic_configuration["epoch"], pop_size=metaheuristic_configuration["pop_size"])
    best_solution = None
    current_iteration = 0
    trainings_epochs.append(
        {"epoch": current_iteration, "trainings": trainings_duration, "best_solution": best_solution})
    trainings_duration = []

{'fit': 0.007356503757696794, 'position': array([4.18433119e+00, 3.67094438e-03, 2.85525768e+00, 4.49312736e+00,
       8.43721361e+00, 6.94062968e+00, 8.41104888e+00])}
Resuming from iteration 3


In [54]:
# Run the optimization loop
# while current_iteration < metaheuristic_configuration["epoch"]:
# Perform GWO iteration
best_position, best_fitness = model.solve()

# Update the best solution
if best_solution is None or best_fitness < best_solution["fit"]:
    best_solution = {
        "fit": best_fitness,
        "position": best_position
    }

current_iteration += 1
trainings_epochs.append({"epoch": current_iteration,
                        "trainings": trainings_duration, "best_solution": best_solution})
trainings_duration = []

# Increment the iteration count
# Save the session periodically
with open(session_file, 'wb') as f:
    pickle.dump({
        "model": model,
        "best_solution": best_solution,
        "current_iteration": current_iteration
    }, f)


###########################################################


--------------------------4/0--------------------------------


###########################################################



Epoch Training Started at : 2023-06-20 19:16:06.984299
---Currently working one voxel : 1, voxels left : 3 
Voxel Training 1 Started at : 2023-06-20 19:16:06.984421
Train on 249 samples
Epoch 1/30
249/249 [==============================] - 5s 18ms/sample - loss: 0.4730 - _calculate_reconstruction_loss: 0.2225 - _calculate_porosity_loss: 0.2120 - _calculate_kl_loss: 35.3959
Epoch 2/30
249/249 [==============================] - 2s 6ms/sample - loss: 0.3192 - _calculate_reconstruction_loss: 0.1190 - _calculate_porosity_loss: 0.1645 - _calculate_kl_loss: 35.0445
Epoch 3/30
249/249 [==============================] - 1s 6ms/sample - loss: 0.2660 - _calculate_reconstruction_loss: 0.0937 - _calculate_porosity_loss: 0.1379 - _calculate_kl_loss: 34.0347
Epoch 4/30
249/249 [==============================] - 2s 6ms/sample - loss: 0.2265 - _calculate_reconstruction_loss: 0.0774 - _calculate_porosity_loss: 0.1153 - _calculate_kl_loss: 33.4120
Epoch 5/30
249/249 [==============================] - 1s 

Epoch Training Started at : 2023-06-20 19:17:14.634881
---Currently working one voxel : 1, voxels left : 3 
Voxel Training 1 Started at : 2023-06-20 19:17:14.635135
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 6s 22ms/sample - loss: 0.4310 - _calculate_reconstruction_loss: 0.1877 - _calculate_porosity_loss: 0.2130 - _calculate_kl_loss: 27.9990
Epoch 2/40
249/249 [==============================] - 2s 8ms/sample - loss: 0.2768 - _calculate_reconstruction_loss: 0.0959 - _calculate_porosity_loss: 0.1516 - _calculate_kl_loss: 28.0322
Epoch 3/40
249/249 [==============================] - 2s 7ms/sample - loss: 0.2045 - _calculate_reconstruction_loss: 0.0649 - _calculate_porosity_loss: 0.1117 - _calculate_kl_loss: 27.6152
Epoch 4/40
249/249 [==============================] - 2s 6ms/sample - loss: 0.1615 - _calculate_reconstruction_loss: 0.0503 - _calculate_porosity_loss: 0.0841 - _calculate_kl_loss: 27.0474
Epoch 5/40
249/249 [==============================] - 2s 

Epoch Training Started at : 2023-06-20 19:18:47.027336
---Currently working one voxel : 1, voxels left : 3 
Voxel Training 1 Started at : 2023-06-20 19:18:47.027471
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 8s 33ms/sample - loss: 0.3757 - _calculate_reconstruction_loss: 0.1522 - _calculate_porosity_loss: 0.1866 - _calculate_kl_loss: 34.2512
Epoch 2/40
249/249 [==============================] - 3s 14ms/sample - loss: 0.2121 - _calculate_reconstruction_loss: 0.0679 - _calculate_porosity_loss: 0.1111 - _calculate_kl_loss: 32.4860
Epoch 3/40
249/249 [==============================] - 3s 13ms/sample - loss: 0.1548 - _calculate_reconstruction_loss: 0.0488 - _calculate_porosity_loss: 0.0721 - _calculate_kl_loss: 33.6492
Epoch 4/40
249/249 [==============================] - 3s 13ms/sample - loss: 0.1230 - _calculate_reconstruction_loss: 0.0397 - _calculate_porosity_loss: 0.0492 - _calculate_kl_loss: 33.8928
Epoch 5/40
249/249 [==============================] - 

Epoch Training Started at : 2023-06-20 19:21:45.157472
---Currently working one voxel : 1, voxels left : 3 
Voxel Training 1 Started at : 2023-06-20 19:21:45.157596
Train on 249 samples
Epoch 1/30
249/249 [==============================] - 13s 51ms/sample - loss: 0.1398 - _calculate_reconstruction_loss: 0.0492 - _calculate_porosity_loss: 0.0398 - _calculate_kl_loss: 49.0423
Epoch 2/30
249/249 [==============================] - 5s 20ms/sample - loss: 0.0690 - _calculate_reconstruction_loss: 0.0278 - _calculate_porosity_loss: 0.0048 - _calculate_kl_loss: 36.1876
Epoch 3/30
249/249 [==============================] - 5s 20ms/sample - loss: 0.0616 - _calculate_reconstruction_loss: 0.0272 - _calculate_porosity_loss: 0.0052 - _calculate_kl_loss: 29.0573
Epoch 4/30
249/249 [==============================] - 5s 21ms/sample - loss: 0.0523 - _calculate_reconstruction_loss: 0.0266 - _calculate_porosity_loss: 0.0045 - _calculate_kl_loss: 21.4683
Epoch 5/30
249/249 [==============================] -

...

Epoch Training Started at : 2023-06-20 23:51:42.753405
---Currently working one voxel : 1, voxels left : 3 
Voxel Training 1 Started at : 2023-06-20 23:51:42.753545
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 8s 31ms/sample - loss: 0.1486 - _calculate_reconstruction_loss: 0.0583 - _calculate_porosity_loss: 0.0609 - _calculate_kl_loss: 27.1270
Epoch 2/40
249/249 [==============================] - 3s 14ms/sample - loss: 0.0554 - _calculate_reconstruction_loss: 0.0300 - _calculate_porosity_loss: 0.0045 - _calculate_kl_loss: 20.6754
Epoch 3/40
249/249 [==============================] - 3s 13ms/sample - loss: 0.0502 - _calculate_reconstruction_loss: 0.0284 - _calculate_porosity_loss: 0.0039 - _calculate_kl_loss: 17.9627
Epoch 4/40
249/249 [==============================] - 3s 13ms/sample - loss: 0.0477 - _calculate_reconstruction_loss: 0.0282 - _calculate_porosity_loss: 0.0039 - _calculate_kl_loss: 15.6519
Epoch 5/40
249/249 [==============================] - 

2023/06/21 12:01:38 AM, INFO, mealpy.swarm_based.GWO.BaseGWO: >Epoch: 5, Current best: 0.0064505320547305674, Global best: 0.0064505320547305674, Runtime: 4531.33982 seconds


Voxel Training 4 Ended at : 2023-06-21 00:01:38.792958 and it took 150.3377559185028s
Epoch Training Ended at : 2023-06-21 00:01:38.793103 and it took 596.0396981239319s


INFO:mealpy.swarm_based.GWO.BaseGWO:>Epoch: 5, Current best: 0.0064505320547305674, Global best: 0.0064505320547305674, Runtime: 4531.33982 seconds


PicklingError: Can't pickle <function objective_function at 0x7f59656f6280>: it's not the same object as __main__.objective_function

In [69]:
model
with open( 'gwo_sessiond.pkl', 'wb') as f:
    pickle.dump({
        "model": model,
        "best_solution": best_solution,
        "current_iteration": current_iteration
    }, f)

PicklingError: Can't pickle <function objective_function at 0x7f59656f6280>: it's not the same object as __main__.objective_function

In [70]:
# print("\n".join( [str(training) for epoch in trainings_epochs for training in epoch]))
print(f"Best solution : {best_solution['fit']}")
batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = decode_solution(
    best_solution["position"])
print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}")

Best solution : 0.0064505320547305674
Batch size : 16, learning_rate : 0.0031913047871977907, opt : Adam, epoch : 40, num_filters : 7, latent_space_dim : 64, reduced_dim : 256


In [74]:
best_solution

{'fit': 0.0064505320547305674,
 'position': array([4.44487183e+00, 3.19130479e-03, 1.72301470e+00, 4.09684848e+00,
        7.84535410e+00, 6.97799716e+00, 8.77545588e+00])}

- Best solution : 0.007356503757696794
- Batch size : 16, learning_rate : 0.003670944375415099, opt : RMSprop, epoch : 40, num_filters : 8, latent_space_dim : 64, reduced_dim : 256

In [71]:
current_iteration

0

### Save History

In [73]:
model.history.save_global_objectives_chart(filename="hello/goc")
model.history.save_local_objectives_chart(filename="hello/loc")

model.history.save_global_best_fitness_chart(filename="hello/gbfc")
model.history.save_local_best_fitness_chart(filename="hello/lbfc")

model.history.save_runtime_chart(filename="hello/rtc")

model.history.save_exploration_exploitation_chart(filename="hello/eec")

model.history.save_diversity_chart(filename="hello/dc")

model.history.save_trajectory_chart(
    list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello/tc")

### Remove old sessions

In [ ]:
if os.path.exists(session_file):
    os.remove(session_file)

In [72]:
print(len(trainings_duration))
trainings_epochs.append({"epoch": current_iteration,
                        "trainings": trainings_duration, "best_solution": best_solution})
trainings_duration = []

0


In [50]:
for training in trainings_epochs:
    print(len(training["trainings"]))

67
0
0
1
61
0


In [51]:
import json
jd = []
for training in trainings_epochs:
    json_data = []
    for obj in training["trainings"]:
        training_obj = obj.__dict__
        samples = []
        for sample in training_obj["samples"]:
            if type(sample) is dict:
                samples.append(sample)
            else :
                samples.append(sample.__dict__)
        training_obj["samples"] = samples
        json_data.append(training_obj)
    jd.append(json_data)

# Print the JSON data
print(jd)
# Convert the object to JSON
with open("output.json", "w") as file:
    file.write(json.dumps(jd))

[[{'title': 'Epoch Training', 'started': 1687218997.5178235, 'params': {'inputShape': (256, 256, 1), 'latent_space_dim': 64, 'reducedDimension': 128, 'num_conv_layers': 5, 'learning_rate': 0.004796511360185954, 'batch_size': 32, 'epochs': 40, 'opt': 'RMSprop', 'finished': True, 'loss': 0.011494855239359012}, 'ended': 1687219266.5109344, 'samples': [{'title': 'Voxel Training 1', 'started': 1687218997.517937, 'params': {}, 'ended': 1687219074.3470802}, {'title': 'Voxel Training 2', 'started': 1687219074.347213, 'params': {}, 'ended': 1687219135.1712325}, {'title': 'Voxel Training 3', 'started': 1687219135.1714194, 'params': {}, 'ended': 1687219199.941699}, {'title': 'Voxel Training 4', 'started': 1687219199.941769, 'params': {}, 'ended': 1687219266.5108666}]}, {'title': 'Epoch Training', 'started': 1687219270.6455877, 'params': {'inputShape': (256, 256, 1), 'latent_space_dim': 64, 'reducedDimension': 512, 'num_conv_layers': 6, 'learning_rate': 0.005165738153087023, 'batch_size': 32, 'epo

## Training Best model


### compile and build

In [77]:
# print("\n".join( [str(training) for epoch in trainings_epochs for training in epoch]))
print(f"Best solution : {best_solution['fit']}")
batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = decode_solution(best_solution["position"])
print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}")

Best solution : 0.0064505320547305674
Batch size : 16, learning_rate : 0.0031913047871977907, opt : Adam, epoch : 40, num_filters : 7, latent_space_dim : 64, reduced_dim : 256


In [81]:
B_S_P=[x for x in best_solution["position"]]
with open("best_solution.json", "w") as file:
    file.write(json.dumps(B_S_P))

In [85]:
# best from model
batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = decode_solution(best_solution["position"])

In [86]:
# best from view
batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = 16, 0.007536762790792937, "Adam", 40, 7, 64 , 512

In [ ]:
# best from view
batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = 16, 0.003670944375415099, "RMSprop", 40, 8, 64 , 512

In [87]:

reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                    latent_space_dim=latent_space_dim,
                                    reducedDimension=reduced_dim,
                                    num_conv_layers=num_filters,
                                    learning_rate=learning_rate,
                                    batch_size=batch_size,
                                    epochs=epoch,
                                    opt=opt,
                                    )
reconstruction.summary()
reconstruction.compile()

Model: "learned_prior"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 learned_prior_input (InputL  [(None, 256, 256, 1)]    0         
 ayer)                                                           
                                                                 
 encoder_conv_1 (Conv2D)     (None, 256, 256, 16)      160       
                                                                 
 batch_normalization_72 (Bat  (None, 256, 256, 16)     64        
 chNormalization)                                                
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 128, 128, 16)     0         
 g2D)                                                            
                                                                 
 encoder_conv_2 (Conv2D)     (None, 128, 128, 32)      4640      
                                                     

In [84]:
params = {
    "inputShape": CONFIGURATION["INPUT_SHAPE"],
    "latent_space_dim": latent_space_dim,
    "reducedDimension": reduced_dim,
    "num_conv_layers": num_filters,
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "epochs": epoch,
    "opt": opt,
    "finished":False
}
trainingTime = TrainingData(f"Epoch Training {current_training}", params)
training_loss=1
for i in range(len(learnedVoxels)):
    print(f"---Currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
    sampleTime = trainingTime.SampleStarted(f"Voxel Training {i+1}")
    try:
        # Code that may raise an OOM error
        training_result=reconstruction.train(learnedVoxels[i], inferenceVoxels[i])
        sampleTime.End()
        training_loss=training_result.history['loss'][-1]
        print(f"training loss for this voxel : {training_loss}")
    except MemoryError as e:
        # Code to handle the OOM error
        trainingTime.setParams("OMM_Errored",str(e))
        break
    except Exception as e:
        trainingTime.setParams("Error",str(e))
        break
    

trainingTime.setParams("loss",training_loss)
trainingTime.setParams("finished",True)
trainingTime.End()

reconstruction.save( save_folder=f"results/model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}")
print(f"training loss for the training : {training_loss}")

Epoch Training 0 Started at : 2023-06-21 00:29:45.195096
---Currently working one voxel : 1, voxels left : 57 
Voxel Training 1 Started at : 2023-06-21 00:29:45.196562
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 8s 32ms/sample - loss: 0.1500 - _calculate_reconstruction_loss: 0.0526 - _calculate_porosity_loss: 0.0666 - _calculate_kl_loss: 28.2585
Epoch 2/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.0568 - _calculate_reconstruction_loss: 0.0286 - _calculate_porosity_loss: 0.0041 - _calculate_kl_loss: 23.9959
Epoch 3/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.0538 - _calculate_reconstruction_loss: 0.0283 - _calculate_porosity_loss: 0.0038 - _calculate_kl_loss: 21.7253
Epoch 4/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.0521 - _calculate_reconstruction_loss: 0.0279 - _calculate_porosity_loss: 0.0033 - _calculate_kl_loss: 20.8310
Epoch 5/40
249/249 [==============================]

In [88]:
training_loss

0.006125778618107838

## Testing the model

### Load test data

In [ ]:
X_test = np.load('X_test.npy')
X_test.shape

In [ ]:
first = X_test[0]
Topredict = first[1:]
test = first[:-1]

In [ ]:
n = 10

plt.figure(figsize=(20, 4))
for i in range(n):
    # original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(Topredict[i].astype("float32"))
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # reconstructed
    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(testing[i].reshape(256, 256, 1))
    plt.title('reconstructed')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
# reconstruction.Generate.save('reconstruction_10batch_10epochs.h5')

In [ ]:
file_path = "reconstructed.raw"
testing.insert(0, first[0].reshape(1, 256, 256, 1))
print(len(testing))

In [ ]:
final = np.array(testing, np.float32).reshape(250, 256, 256)
final.shape

In [ ]:
file_path = "my_array.npy"
np.save("file", final)

In [ ]:
final.tofile(file_path)

In [ ]:
Volume = np.fromfile(file_path, dtype=np.uint8)
Volume = Volume.reshape(250, 256, 256)
Volume.shape